# This is a Demo of processing Video footage and determine if it contains security concern using Alex net, a convolutional neural network ML models

This model suitable for processing image input. As some anomaly videos only need few frame/photo to determine assualting or other security concerns happening.

Thus, we will process videos into 8 frames as a input, so we have contiuous story.

The datasets from center for research of computer vision
https://www.crcv.ucf.edu/projects/real-world/

Real-world Anomaly Detection in Surveillance Videos

https://www.dropbox.com/scl/fo/2aczdnx37hxvcfdo4rq4q/AOjRokSTaiKxXmgUyqdcI6k?rlkey=5bg7mxxbq46t7aujfch46dlvz&e=1&dl=0

Datasets chose:

1. Anomaly-Videos-Part-1
  consist 4 subcategories: Abuse, arrest, arson, and assualt.
  Each subCate has 50 videos, total 200 anomaly footage

2. Testing_Normal_Videos
  consist 200 normal footages
3. A model never seen senario(Bulgary) to test model' s generalizaiton ability.

videos across different scenario: Foregin street, supermarket, parking lot etc.



# Prepare the dataset, mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os

base_zip_dir = "/content/drive/MyDrive/SecurityRiskDetector"
raw_root     = "/content/data/raw"

os.makedirs(raw_root, exist_ok=True)
print(os.listdir(base_zip_dir))


Mounted at /content/drive
['Testing_Normal_Videos.zip', 'Anomaly-Videos-Part-1.zip', 'frames_v1', 'preprocessDataset.ipynb', 'Anomaly-Videos-Part-2.zip', 'V2preprocessDataset.ipynb', 'V3preprocessDataset.ipynb']


In [ ]:
# Unzip datasets
!mkdir -p /content/data/raw

!unzip -q "/content/drive/MyDrive/SecurityRiskDetector/Anomaly-Videos-Part-1.zip"   -d "/content/data/raw/anomaly"
!unzip -q "/content/drive/MyDrive/SecurityRiskDetector/Anomaly-Videos-Part-2.zip"   -d "/content/data/raw/anomaly"
!unzip -q "/content/drive/MyDrive/SecurityRiskDetector/Testing_Normal_Videos.zip"   -d "/content/data/raw/normal"

In [ ]:
!# See top-level raw folder
!ls -R /content/data/raw


/content/data/raw:
anomaly  normal

/content/data/raw/anomaly:
Anomaly-Videos-Part-1  Anomaly-Videos-Part-2

/content/data/raw/anomaly/Anomaly-Videos-Part-1:
Abuse  Arrest  Arson  Assault

/content/data/raw/anomaly/Anomaly-Videos-Part-1/Abuse:
Abuse001_x264.mp4  Abuse014_x264.mp4  Abuse027_x264.mp4  Abuse040_x264.mp4
Abuse002_x264.mp4  Abuse015_x264.mp4  Abuse028_x264.mp4  Abuse041_x264.mp4
Abuse003_x264.mp4  Abuse016_x264.mp4  Abuse029_x264.mp4  Abuse042_x264.mp4
Abuse004_x264.mp4  Abuse017_x264.mp4  Abuse030_x264.mp4  Abuse043_x264.mp4
Abuse005_x264.mp4  Abuse018_x264.mp4  Abuse031_x264.mp4  Abuse044_x264.mp4
Abuse006_x264.mp4  Abuse019_x264.mp4  Abuse032_x264.mp4  Abuse045_x264.mp4
Abuse007_x264.mp4  Abuse020_x264.mp4  Abuse033_x264.mp4  Abuse046_x264.mp4
Abuse008_x264.mp4  Abuse021_x264.mp4  Abuse034_x264.mp4  Abuse047_x264.mp4
Abuse009_x264.mp4  Abuse022_x264.mp4  Abuse035_x264.mp4  Abuse048_x264.mp4
Abuse010_x264.mp4  Abuse023_x264.mp4  Abuse036_x264.mp4  Abuse049_x264.mp4
Abuse0

In [ ]:
from pathlib import Path

anomaly_root = Path("/content/data/raw/anomaly/Anomaly-Videos-Part-1")
anomaly_root2 = Path("/content/data/raw/anomaly/Anomaly-Videos-Part-1")
normal_root  = Path("/content/data/raw/normal/Testing_Normal_Videos_Anomaly")

# grab all mp4s recursively
anomaly_videos = sorted(anomaly_root.rglob("*.mp4"))
anomaly_videos2 = sorted(anomaly_root2.rglob("*.mp4"))
normal_videos  = sorted(normal_root.rglob("*.mp4"))


print("Anomaly video count:", len(anomaly_videos))
print("Anomaly video count:", len(anomaly_videos2))
print("Normal video count :", len(normal_videos))

# peek a few paths
print("Sample anomaly:", anomaly_videos[:3])
print("Sample anomaly2:", anomaly_videos2[:3])
print("Sample normal :", normal_videos[:3])


Anomaly video count: 200
Anomaly video count: 200
Normal video count : 150
Sample anomaly: [PosixPath('/content/data/raw/anomaly/Anomaly-Videos-Part-1/Abuse/Abuse001_x264.mp4'), PosixPath('/content/data/raw/anomaly/Anomaly-Videos-Part-1/Abuse/Abuse002_x264.mp4'), PosixPath('/content/data/raw/anomaly/Anomaly-Videos-Part-1/Abuse/Abuse003_x264.mp4')]
Sample anomaly2: [PosixPath('/content/data/raw/anomaly/Anomaly-Videos-Part-1/Abuse/Abuse001_x264.mp4'), PosixPath('/content/data/raw/anomaly/Anomaly-Videos-Part-1/Abuse/Abuse002_x264.mp4'), PosixPath('/content/data/raw/anomaly/Anomaly-Videos-Part-1/Abuse/Abuse003_x264.mp4')]
Sample normal : [PosixPath('/content/data/raw/normal/Testing_Normal_Videos_Anomaly/Normal_Videos_003_x264.mp4'), PosixPath('/content/data/raw/normal/Testing_Normal_Videos_Anomaly/Normal_Videos_006_x264.mp4'), PosixPath('/content/data/raw/normal/Testing_Normal_Videos_Anomaly/Normal_Videos_010_x264.mp4')]


# extract frames as training input

video wise training input is too big for my local machine.

(a 10 mins long, 30 FPS video can have 10min * 60seconds * 30 frames = 18000 photos, way too big to train on single GPU)

I extract 16 frames from a videos regardless their duration save as photo to capture global info.




In [ ]:
import cv2
from pathlib import Path

FRAME_ROOT = Path("/content/data/frames")
FRAME_ROOT.mkdir(parents=True, exist_ok=True)

# adjust frames to capture more info
N_FRAMES = 16

def extract_frames_from_video(video_path: Path, out_dir: Path, n_frames: int = N_FRAMES):
    cap = cv2.VideoCapture(str(video_path))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_count <= 0:
        cap.release()
        return

    step = max(frame_count // n_frames, 1)
    indices = list(range(0, frame_count, step))[:n_frames]

    out_dir.mkdir(parents=True, exist_ok=True)
    i = 0
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            continue
        out_path = out_dir / f"{video_path.stem}_frame{i:03d}.jpg"
        cv2.imwrite(str(out_path), frame)
        i += 1

    cap.release()

# ---- extract anomaly frames ----
for vfile in anomaly_videos:
    extract_frames_from_video(vfile, FRAME_ROOT / "anomaly")

# ---- extract anomaly2 frames ----
for vfile2 in anomaly_videos2:
    extract_frames_from_video(vfile2, FRAME_ROOT / "anomaly2")

# ---- extract normal frames ----
for vfile in normal_videos:
    extract_frames_from_video(vfile, FRAME_ROOT / "normal")
import glob

print("Anomaly frames:", len(glob.glob("/content/data/frames/anomaly/*.jpg")))
print("Anomaly frames2:", len(glob.glob("/content/data/frames/anomaly2/*.jpg")))
print("Normal frames :", len(glob.glob("/content/data/frames/normal/*.jpg")))


Anomaly frames: 3200
Anomaly frames2: 3200
Normal frames : 2400


In [ ]:
# Save preprocessed dataset locally

!mkdir -p "/content/drive/MyDrive/SecurityRiskDetector/frames_v1"
!cp -r /content/data/frames/* "/content/drive/MyDrive/SecurityRiskDetector/frames_v1/"


# Train a model

In [ ]:
# !pip install torch torchvision


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import os
import numpy as np

# FRAME_ROOT = "/content/data/frames"  # anomaly / normal live here
FRAME_ROOT = "/content/drive/MyDrive/SecurityRiskDetector/frames_v1"
class FrameDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        classes = [("normal", 0), ("anomaly", 1)]
        for cls_name, label in classes:
            folder = os.path.join(root_dir, cls_name)
            for fname in os.listdir(folder):
                if fname.lower().endswith((".jpg", ".png", ".jpeg")):
                    self.samples.append((os.path.join(folder, fname), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

dataset = FrameDataset(FRAME_ROOT, transform=transform)
len(dataset)


5600

In [ ]:
from collections import Counter

labels = [lbl for _, lbl in dataset]
print("Label counts:", Counter(labels))


Label counts: Counter({1: 3200, 0: 2400})


In [ ]:
n_total = len(dataset)
# n_train = int(0.8 * n_total)
n_train = int(0.5 * n_total)
n_val = n_total - n_train

train_ds, val_ds = torch.utils.data.random_split(dataset, [n_train, n_val])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, num_workers=2)

len(train_ds), len(val_ds)


(2800, 2800)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [ ]:
from torchvision import models

model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)  # pretrained
model.fc = torch.nn.Linear(model.fc.in_features, 2)  # 2 classes: normal/anomaly
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 97.3MB/s]


In [ ]:
def train_epoch(loader):
    model.train()
    total_loss, correct, total = 0.0, 0, 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * imgs.size(0)
        preds = outputs.argmax(1)
        correct += (preds == labels).sum().item()
        total += imgs.size(0)

    return total_loss / total, correct / total

@torch.no_grad()
def eval_epoch(loader):
    model.eval()
    total_loss, correct, total = 0.0, 0, 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        loss = criterion(outputs, labels)

        total_loss += loss.item() * imgs.size(0)
        preds = outputs.argmax(1)
        correct += (preds == labels).sum().item()
        total += imgs.size(0)

    return total_loss / total, correct / total


In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
    tr_loss, tr_acc = train_epoch(train_loader)
    val_loss, val_acc = eval_epoch(val_loader)
    print(f"Epoch {epoch+1}/{EPOCHS} "
          f"- train_loss: {tr_loss:.4f}, train_acc: {tr_acc:.3f}, "
          f"val_loss: {val_loss:.4f}, val_acc: {val_acc:.3f}")


Epoch 1/5 - train_loss: 0.1383, train_acc: 0.944, val_loss: 0.0240, val_acc: 0.993
Epoch 2/5 - train_loss: 0.0102, train_acc: 0.999, val_loss: 0.0111, val_acc: 0.998
Epoch 3/5 - train_loss: 0.0072, train_acc: 0.999, val_loss: 0.0127, val_acc: 0.995
Epoch 4/5 - train_loss: 0.0092, train_acc: 0.998, val_loss: 0.0165, val_acc: 0.994
Epoch 5/5 - train_loss: 0.0045, train_acc: 0.998, val_loss: 0.0230, val_acc: 0.995


# Test a single video

In [ ]:
import cv2
import torch.nn.functional as F
from pathlib import Path
from PIL import Image

N_FRAMES_TEST = 16  # same idea as training

def predict_video_binary(video_path, model, transform, device, n_frames=N_FRAMES_TEST):
    video_path = str(video_path)
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_count <= 0:
        cap.release()
        raise ValueError(f"Cannot read video or video empty: {video_path}")

    step = max(frame_count // n_frames, 1)
    indices = list(range(0, frame_count, step))[:n_frames]

    frames = []
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            continue
        # OpenCV reads BGR, convert to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img = transform(img)
        frames.append(img)

    cap.release()

    if len(frames) == 0:
        raise ValueError(f"No frames extracted from: {video_path}")

    batch = torch.stack(frames).to(device)  # [T, 3, 224, 224]

    model.eval()
    with torch.no_grad():
        outputs = model(batch)             # [T, 2]
        probs = F.softmax(outputs, dim=1)  # [T, 2]
        mean_probs = probs.mean(dim=0)     # [2]

    prob_normal  = mean_probs[0].item()
    prob_anomaly = mean_probs[1].item()
    pred_label   = 1 if prob_anomaly > 0.5 else 0  # 1 = anomaly, 0 = normal

    return {
        "pred_label": pred_label,
        "prob_normal": prob_normal,
        "prob_anomaly": prob_anomaly,
        "video_path": video_path,
    }


In [ ]:
from pathlib import Path

test_video = Path("/content/data/raw/anomaly/Anomaly-Videos-Part-1/Abuse/Abuse001_x264.mp4")

result = predict_video_binary(test_video, model, transform, device)
result


{'pred_label': 1,
 'prob_normal': 0.0004482882795855403,
 'prob_anomaly': 0.9995517134666443,
 'video_path': '/content/data/raw/anomaly/Anomaly-Videos-Part-1/Abuse/Abuse001_x264.mp4'}

In [ ]:
test_video_normal = Path("/content/data/raw/normal/Testing_Normal_Videos_Anomaly/Normal_Videos_003_x264.mp4")

result_normal = predict_video_binary(test_video_normal, model, transform, device)
result_normal


{'pred_label': 0,
 'prob_normal': 0.9999719262123108,
 'prob_anomaly': 2.8109898266848177e-05,
 'video_path': '/content/data/raw/normal/Testing_Normal_Videos_Anomaly/Normal_Videos_003_x264.mp4'}

In [20]:
import random

# small sample to avoid running forever
sample_anomaly = random.sample(anomaly_videos, 10)
sample_normal  = random.sample(normal_videos, 10)

correct = 0
total = 0

for v in sample_anomaly:
    res = predict_video_binary(v, model, transform, device)
    if res["pred_label"] == 1:
        correct += 1
    total += 1


for v in sample_normal:
    res = predict_video_binary(v, model, transform, device)
    if res["pred_label"] == 0:
        correct += 1
    total += 1

print(f"Video-level accuracy on 20 samples: {correct}/{total} = {correct/total:.3f}")


Video-level accuracy on 20 samples: 20/20 = 1.000


## test on other scenario, this case Burglary

In [21]:
from pathlib import Path

test_video = Path("/content/Burglary003_x264.mp4")

result = predict_video_binary(test_video, model, transform, device)
result


{'pred_label': 1,
 'prob_normal': 0.0037782767321914434,
 'prob_anomaly': 0.9962217807769775,
 'video_path': '/content/Burglary003_x264.mp4'}



# Limitations and future directions:

This model is for DEMO only. for simplicity and limited training time and resources(I only have a few hours of GPU usage on Cloud), I extract a fixed number of images from a video and make a determination purely based on these images. It can not capture motion, and bad extraction can not capture essential security images which resulting in false negatives(unable to determine its anomaly).

It ideally uses recurrent or transformer models such as LSTM and attention models. So it can process video-flow data, and is able to capture spatial and time features, pay attention(sensitive) to specific details of an image/video-cut, such as weapons and fires.

To avoid false alarms while keeping a low false negative rate(False negative is crucial than accuracy for safety concerns), we may consider using multiple dense and complex models to process the video and make an ensemble vote.